In [22]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import os

import haversine as hs

from datetime import datetime, timedelta

import dateutil.parser

import random

In [2]:
from IPython.display import display

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [189]:
flyaway = pd.read_csv("Circuits_Flyaway.csv")

flyaway.shape

(24, 9)

In [190]:
flyaway

,country,location,circuit,circuitRef,lat,lng,raceId,circuitId,type
0,Bahrain,Sakhir,Bahrain International Circuit,bahrain,26.0325,50.51060,1098,3,flyaway
1,Saudi Arabia,Jeddah,Jeddah Corniche Circuit,jeddah,21.6319,39.10440,1099,77,flyaway
2,Australia,Melbourne,Albert Park Grand Prix Circuit,albert_park,-37.8497,144.96800,1100,1,flyaway
3,China,Shanghai,Shanghai International Circuit,shanghai,31.3389,121.22000,1012,17,flyaway
4,Azerbaijan,Baku,Baku City Circuit,baku,40.3725,49.85330,1101,73,flyaway
5,USA,Miami,Miami International Autodrome,miami,25.9581,-80.23890,1102,79,flyaway
6,Italy,Imola,Autodromo Enzo e Dino Ferrari,imola,44.3439,11.71670,1077,21,road
7,Monaco,Monte-Carlo,Circuit de Monaco,monaco,43.7347,7.42056,1104,6,road
8,Spain,Montmeló,Circuit de Barcelona-Catalunya,catalunya,41.5700,2.26111,1105,4,road
9,Canada,Montreal,Circuit Gilles Villeneuve,villeneuve,45.5000,-73.52280,1106,7,flyaway


In [191]:
flyaway['type'].value_counts()

flyaway    15
road        9
Name: type, dtype: int64

In [192]:
ships = [-1, -1, -1, -1, -1]

In [4]:
distances = pd.read_csv("Circuit_Distances.csv", index_col=0)

distances.shape

(24, 24)

In [5]:
distances

,bahrain,jeddah,albert_park,shanghai,baku,miami,imola,monaco,catalunya,villeneuve,red_bull_ring,silverstone,hungaroring,spa,zandvoort,monza,marina_bay,suzuka,losail,americas,rodriguez,interlagos,vegas,yas_marina
bahrain,0.00,1258.43,12112.63,6804.82,1595.70,12185.63,4018.27,4332.65,4710.93,10258.97,3910.84,5158.08,3628.99,4640.39,4805.05,4242.73,6327.19,8054.34,112.11,12908.78,13990.13,11813.59,12943.28,446.84
jeddah,1258.43,0.00,12817.12,8062.08,2317.68,11605.63,3559.50,3810.52,4087.37,9929.43,3585.08,4815.75,3387.79,4307.69,4515.10,3797.55,7353.82,9293.32,1329.13,12632.62,13574.58,10555.63,13047.49,1615.85
albert_park,12112.63,12817.12,0.00,8078.46,12989.08,15594.50,16086.36,16422.24,16823.35,16741.10,15878.75,16947.24,15546.43,16511.75,16572.56,16288.08,6057.69,8129.72,12000.56,14286.08,13563.81,13062.93,13131.15,11674.76
shanghai,6804.82,8062.08,8078.46,0.00,6336.88,13248.55,8986.37,9302.00,9779.98,11342.35,8611.28,9175.17,8302.58,8926.20,8867.67,9058.02,3806.81,1477.81,6741.58,12043.14,12920.89,18553.96,10488.07,6494.10
baku,1595.70,2317.68,12989.08,6336.88,0.00,11015.93,3135.79,3484.89,3946.50,8930.53,2890.55,4030.59,2557.38,3545.36,3652.58,3314.42,6946.64,7342.55,1661.51,11489.38,12631.93,12217.83,11373.52,1823.13
miami,12185.63,11605.63,15594.50,13248.55,11015.93,0.00,8172.92,7870.83,7536.29,2253.88,8278.97,7043.58,8573.80,7556.53,7408.92,7945.20,16953.20,12224.22,12295.53,1767.69,2064.12,6596.17,3492.74,12600.10
imola,4018.27,3559.50,16086.36,8986.37,3135.79,8172.92,0.00,349.98,828.38,6372.27,397.58,1273.39,683.89,803.27,1038.66,238.09,10077.87,9598.52,4129.25,9074.91,10054.68,9612.48,9592.05,4443.94
monaco,4332.65,3810.52,16422.24,9302.00,3484.89,7870.83,349.98,0.00,485.64,6121.74,690.95,1119.23,1012.39,753.28,985.59,255.69,10425.06,9874.96,4444.17,8824.30,9778.21,9306.38,9413.94,4763.03
catalunya,4710.93,4087.37,16823.35,9779.98,3946.50,7536.29,828.38,485.64,0.00,5891.51,1173.27,1194.50,1497.97,1026.45,1215.20,722.01,10873.36,10323.61,4822.99,8582.43,9487.42,8834.87,9288.44,5148.63
villeneuve,10258.97,9929.43,16741.10,11342.35,8930.53,2253.88,6372.27,6121.74,5891.51,0.00,6390.49,5137.22,6644.71,5655.01,5470.23,6134.60,14805.77,10584.02,10362.82,2703.19,3731.54,8159.68,3612.76,10635.91


In [63]:
windows = pd.read_csv("Track_Windows.csv")

windows.shape

(140, 2)

In [64]:
windows

,month,circuitRef
0,3,albert_park
1,3,americas
2,3,bahrain
3,3,catalunya
4,3,interlagos
5,3,jeddah
6,3,losail
7,3,marina_bay
8,3,miami
9,3,rodriguez


In [144]:
windows_copy = windows.copy()

windows_copy.shape

(140, 2)

In [145]:
windows_copy

,month,circuitRef
0,3,albert_park
1,3,americas
2,3,bahrain
3,3,catalunya
4,3,interlagos
5,3,jeddah
6,3,losail
7,3,marina_bay
8,3,miami
9,3,rodriguez


In [30]:
schedule = {3:2, 4:3, 5:3, 6:2, 7:4, 8:1, 9:3, 10:3, 11:3}

print(schedule)

{3: 2, 4: 3, 5: 3, 6: 2, 7: 4, 8: 1, 9: 3, 10: 3, 11: 3}


In [157]:
race = 1
window_cols = ['month', 'round', 'circuitRef']
circuit_windows = dict.fromkeys(window_cols, 0)
race_windows = pd.DataFrame(columns = window_cols)

for i in schedule.keys():
    
    for j in range(1, schedule[i]+1):
        
        print(f"Round {race:<2} | Month {i:<2} : Weekend {j:<2}\n")
        print(windows_copy.loc[windows_copy['month'] == i, 'circuitRef'].values.tolist())
        print()
        
        
        for circuit in windows_copy.loc[windows_copy['month'] == i, 'circuitRef'].values.tolist():
                circuit_windows = dict.fromkeys(window_cols, 0)
                circuit_windows['month'] = i
                circuit_windows['round'] = race
                circuit_windows['circuitRef'] = circuit
                winds = pd.DataFrame([circuit_windows])
                race_windows = pd.concat([race_windows, winds], ignore_index=True)
            
        race += 1

Round 1  | Month 3  : Weekend 1 

['albert_park', 'americas', 'bahrain', 'catalunya', 'interlagos', 'jeddah', 'losail', 'marina_bay', 'miami', 'rodriguez', 'shanghai', 'suzuka', 'vegas', 'yas_marina']

Round 2  | Month 3  : Weekend 2 

['albert_park', 'americas', 'bahrain', 'catalunya', 'interlagos', 'jeddah', 'losail', 'marina_bay', 'miami', 'rodriguez', 'shanghai', 'suzuka', 'vegas', 'yas_marina']

Round 3  | Month 4  : Weekend 1 

['albert_park', 'americas', 'bahrain', 'baku', 'catalunya', 'hungaroring', 'imola', 'interlagos', 'marina_bay', 'miami', 'monza', 'rodriguez', 'shanghai', 'silverstone', 'suzuka', 'vegas', 'zandvoort']

Round 4  | Month 4  : Weekend 2 

['albert_park', 'americas', 'bahrain', 'baku', 'catalunya', 'hungaroring', 'imola', 'interlagos', 'marina_bay', 'miami', 'monza', 'rodriguez', 'shanghai', 'silverstone', 'suzuka', 'vegas', 'zandvoort']

Round 5  | Month 4  : Weekend 3 

['albert_park', 'americas', 'bahrain', 'baku', 'catalunya', 'hungaroring', 'imola', 'int

In [158]:
race_windows.shape

(378, 3)

In [159]:
race_windows

,month,round,circuitRef
0,3,1,albert_park
1,3,1,americas
2,3,1,bahrain
3,3,1,catalunya
4,3,1,interlagos
5,3,1,jeddah
6,3,1,losail
7,3,1,marina_bay
8,3,1,miami
9,3,1,rodriguez


In [161]:
non_race_windows = pd.read_csv("Track_Windows_v2.csv")

non_race_windows.shape

(50, 3)

In [162]:
non_race_windows

,month,round,circuitRef
0,4,3,bahrain
1,4,4,bahrain
2,11,22,bahrain
3,10,19,shanghai
4,4,3,baku
5,4,4,baku
6,8,15,miami
7,9,16,miami
8,9,17,miami
9,9,18,miami


In [166]:
race_viable = race_windows.merge(non_race_windows.drop_duplicates(), on=['month','round', 'circuitRef'], how='left', indicator=True)

race_viable

,month,round,circuitRef,_merge
0,3,1,albert_park,left_only
1,3,1,americas,both
2,3,1,bahrain,left_only
3,3,1,catalunya,left_only
4,3,1,interlagos,left_only
5,3,1,jeddah,left_only
6,3,1,losail,left_only
7,3,1,marina_bay,left_only
8,3,1,miami,left_only
9,3,1,rodriguez,left_only


In [172]:
weekend_options = race_viable.loc[race_viable['_merge'] == 'left_only']

weekend_options.drop(['_merge'], inplace = True, axis = 1)

weekend_options

,month,round,circuitRef
0,3,1,albert_park
2,3,1,bahrain
3,3,1,catalunya
4,3,1,interlagos
5,3,1,jeddah
6,3,1,losail
7,3,1,marina_bay
8,3,1,miami
9,3,1,rodriguez
10,3,1,shanghai


In [160]:
for i in schedule.keys():
    print(i, race_windows.loc[race_windows['month'] == i].shape[0])

3 28
4 51
5 54
6 32
7 56
8 13
9 51
10 54
11 39


In [31]:
race = 1

for i in schedule.keys():
    
    for j in range(1, schedule[i]+1):
        
        print(f"Round {race:<2} | Month {i:<2} : Weekend {j:<2}\n")
        race += 1

Round 1  | Month 3  : Weekend 1 

Round 2  | Month 3  : Weekend 2 

Round 3  | Month 4  : Weekend 1 

Round 4  | Month 4  : Weekend 2 

Round 5  | Month 4  : Weekend 3 

Round 6  | Month 5  : Weekend 1 

Round 7  | Month 5  : Weekend 2 

Round 8  | Month 5  : Weekend 3 

Round 9  | Month 6  : Weekend 1 

Round 10 | Month 6  : Weekend 2 

Round 11 | Month 7  : Weekend 1 

Round 12 | Month 7  : Weekend 2 

Round 13 | Month 7  : Weekend 3 

Round 14 | Month 7  : Weekend 4 

Round 15 | Month 8  : Weekend 1 

Round 16 | Month 9  : Weekend 1 

Round 17 | Month 9  : Weekend 2 

Round 18 | Month 9  : Weekend 3 

Round 19 | Month 10 : Weekend 1 

Round 20 | Month 10 : Weekend 2 

Round 21 | Month 10 : Weekend 3 

Round 22 | Month 11 : Weekend 1 

Round 23 | Month 11 : Weekend 2 

Round 24 | Month 11 : Weekend 3 



In [256]:
race = 1
loc = ''
parsed = []

cal_cols = range(1, 25)
cal = dict.fromkeys(cal_cols, 0)

start_loc = windows.loc[windows['month'] == 3, 'circuitRef'].values.tolist()
start = random.sample(start_loc, 1)

for start in start_loc:
    cal = dict.fromkeys(cal_cols, 0)
    loc = start
    parsed = []
    race = 1
    total_dist = 0
    #print(start)
    for i in schedule.keys():
        for j in range(1, schedule[i]+1):
            #print(race, loc)
            
            cal[race] = loc
            
            parsed.append(loc)
            
            if j == schedule[i]:
                change = True
            else:
                change = False
            
            if race == 24:
                break
            
            race +=1
            loc, dist = nearest(loc, i, change, parsed)
            total_dist = total_dist + dist
    
    for calendar in cal:
        print(calendar, cal[calendar])
    print("Total distance :", round(total_dist, 2), "kms")
    print()        

        

### ['jeddah']
### ['losail', 'miami', 'yas_marina']
### ['losail', 'yas_marina']
### ['yas_marina']
1 albert_park
2 jeddah
3 bahrain
4 baku
5 hungaroring
6 red_bull_ring
7 imola
8 monza
9 monaco
10 catalunya
11 spa
12 zandvoort
13 silverstone
14 villeneuve
15 rodriguez
16 interlagos
17 marina_bay
18 shanghai
19 suzuka
20 vegas
21 americas
22 miami
23 losail
24 yas_marina
Total distance : 84502.59 kms

### ['jeddah']
### ['losail', 'miami', 'yas_marina']
### ['losail', 'yas_marina']
### ['yas_marina']
1 americas
2 jeddah
3 bahrain
4 baku
5 hungaroring
6 red_bull_ring
7 imola
8 monza
9 monaco
10 catalunya
11 spa
12 zandvoort
13 silverstone
14 villeneuve
15 rodriguez
16 interlagos
17 albert_park
18 marina_bay
19 shanghai
20 suzuka
21 vegas
22 miami
23 losail
24 yas_marina
Total distance : 87421.54 kms

### ['jeddah']
### ['losail', 'miami', 'yas_marina']
### ['losail', 'yas_marina']
### ['yas_marina']
1 bahrain
2 jeddah
3 baku
4 hungaroring
5 imola
6 monza
7 monaco
8 catalunya
9 spa
10 z

In [255]:
def nearest(loc, month, change, parsed):
    if len(parsed) == 0:
        parsed.append(loc)
    
    if change == True:
        month = month + 1
    
    urgent = []
    
    temp = windows.loc[windows['month'] == month, 'circuitRef'].values.tolist()
    
    options = [x for x in temp if x not in parsed]
    
    #return parsed, options
    
    for option in options:
            left = len(windows.loc[(windows['circuitRef'] == option) & (windows['month'] >= month)])
            if left == 1:
                urgent.append(option)
                #next_loc = option
                #return next_loc, distances.loc[loc, next_loc]
    
    #return urgent
    
    if len(urgent) > 0:
        options = urgent
        print("###", urgent)
        
    next_loc = distances.loc[loc, options].sort_values().head(1).index[0]
    dist = distances.loc[loc, options].sort_values().head(1).values[0]
    
    return next_loc, dist
    
    #dists = distances.loc[loc, options].sort_values().head(2).tail(1)

In [132]:
nearest('jeddah', 3, False, [])

('bahrain', 1258.43)

In [133]:
nearest('albert_park', 3, False, ['albert_park'])

('jeddah', 12817.12)

In [81]:
options = windows.loc[windows['month'] == 3, 'circuitRef'].values.tolist()

print(options)
print()

distances.loc['bahrain', options].sort_values()
#print(distances.loc['bahrain', options].sort_values().head(2).tail(1))

['albert_park', 'americas', 'bahrain', 'catalunya', 'interlagos', 'jeddah', 'losail', 'marina_bay', 'miami', 'rodriguez', 'shanghai', 'suzuka', 'vegas', 'yas_marina']



bahrain            0.00
losail           112.11
yas_marina       446.84
jeddah          1258.43
catalunya       4710.93
marina_bay      6327.19
shanghai        6804.82
suzuka          8054.34
interlagos     11813.59
albert_park    12112.63
miami          12185.63
americas       12908.78
vegas          12943.28
rodriguez      13990.13
Name: bahrain, dtype: float64

In [65]:
windows.loc[(windows['circuitRef'] == 'jeddah') & (windows['month'] >= 3)]

,month,circuitRef
5,3,jeddah


In [67]:
distances.loc['bahrain', 'jeddah']

1258.43

In [143]:
race = 1
loc = ''
parsed = []

cal_cols = range(1, 25)
cal = dict.fromkeys(cal_cols, 0)

start_loc = windows_copy.loc[windows_copy['month'] == 3, 'circuitRef'].values.tolist()
start = random.sample(start_loc, 1)

for start in start_loc:
    cal = dict.fromkeys(cal_cols, 0)
    loc = start
    parsed = []
    race = 1
    total_dist = 0
    #print(start)
    for i in schedule.keys():
        for j in range(1, schedule[i]+1):
            #print(race, loc)
            
            cal[race] = loc
            
            parsed.append(loc)
            
            if j == schedule[i]:
                change = True
            else:
                change = False
            
            if race == 24:
                break
            
            race +=1
            loc, dist = nearest_copy(loc, i, change, parsed)
            total_dist = total_dist + dist
    
    for calendar in cal:
        print(calendar, cal[calendar])
    print("Total distance :", round(total_dist, 2), "kms")
    print()        

        

1 albert_park
2 jeddah
3 baku
4 hungaroring
5 imola
6 miami
7 americas
8 rodriguez
9 villeneuve
10 silverstone
11 zandvoort
12 spa
13 monza
14 monaco
15 catalunya
16 red_bull_ring
17 shanghai
18 suzuka
19 marina_bay
20 vegas
21 interlagos
22 bahrain
23 losail
24 yas_marina
Total distamce : 92905.92 kms

1 americas
2 jeddah
3 baku
4 hungaroring
5 imola
6 miami
7 rodriguez
8 villeneuve
9 silverstone
10 zandvoort
11 spa
12 monza
13 monaco
14 catalunya
15 red_bull_ring
16 shanghai
17 suzuka
18 marina_bay
19 albert_park
20 interlagos
21 vegas
22 bahrain
23 losail
24 yas_marina
Total distamce : 97846.44 kms

1 bahrain
2 jeddah
3 baku
4 hungaroring
5 imola
6 miami
7 americas
8 rodriguez
9 villeneuve
10 silverstone
11 zandvoort
12 spa
13 monza
14 monaco
15 catalunya
16 red_bull_ring
17 shanghai
18 suzuka
19 marina_bay
20 albert_park
21 interlagos
22 vegas
23 losail
24 yas_marina
Total distamce : 87344.93 kms

1 catalunya
2 jeddah
3 baku
4 hungaroring
5 imola
6 miami
7 americas
8 rodriguez
9 vi

In [142]:
def nearest_copy(loc, month, change, parsed):
    if len(parsed) == 0:
        parsed.append(loc)
    
    if change == True:
        month = month + 1
    
    urgent = []
    
    temp = windows_copy.loc[windows_copy['month'] == month, 'circuitRef'].values.tolist()
    
    options = [x for x in temp if x not in parsed]
    
    #return parsed, options
    
    for option in options:
            left = len(windows_copy.loc[(windows_copy['circuitRef'] == option) & (windows_copy['month'] >= month)])
            if left == 1:
                urgent.append(option)
                #next_loc = option
                #return next_loc, distances.loc[loc, next_loc]
    
    #return urgent
    
    if len(urgent) > 0:
        options = urgent
        
    next_loc = distances.loc[loc, options].sort_values().head(1).index[0]
    dist = distances.loc[loc, options].sort_values().head(1).values[0]
    
    return next_loc, dist
    
    #dists = distances.loc[loc, options].sort_values().head(2).tail(1)

In [173]:
weekend_options

,month,round,circuitRef
0,3,1,albert_park
2,3,1,bahrain
3,3,1,catalunya
4,3,1,interlagos
5,3,1,jeddah
6,3,1,losail
7,3,1,marina_bay
8,3,1,miami
9,3,1,rodriguez
10,3,1,shanghai


In [175]:
weekend_options['round'].unique().tolist()

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24]

In [258]:
loc = ''
parsed = []

cal_cols = range(1, 25)
cal = dict.fromkeys(cal_cols, 0)

start_loc = weekend_options.loc[weekend_options['round'] == 1, 'circuitRef'].values.tolist()

for start in start_loc:
    cal = dict.fromkeys(cal_cols, 0)
    loc = start
    parsed = []
    total_dist = 0
    
    for i in weekend_options['round'].unique().tolist():
        cal[i] = loc
            
        parsed.append(loc)
            
        if i == 24:
            break
            
        loc, dist = nearest_weekend(loc, i, parsed)
        total_dist = total_dist + dist
    
    for calendar in cal:
        print(calendar, cal[calendar])
    print("Total distance :", round(total_dist, 2), "kms")
    print()        

        

['americas', 'bahrain', 'catalunya', 'interlagos', 'jeddah', 'losail', 'marina_bay', 'miami', 'rodriguez', 'shanghai', 'vegas', 'yas_marina']
### ['jeddah'] 2
['americas', 'catalunya', 'hungaroring', 'imola', 'interlagos', 'marina_bay', 'miami', 'monza', 'rodriguez', 'shanghai', 'silverstone', 'suzuka', 'vegas', 'zandvoort']
['catalunya', 'imola', 'interlagos', 'marina_bay', 'miami', 'monza', 'rodriguez', 'shanghai', 'silverstone', 'suzuka', 'vegas', 'zandvoort']
['americas', 'bahrain', 'baku', 'catalunya', 'marina_bay', 'miami', 'monza', 'rodriguez', 'shanghai', 'silverstone', 'vegas']
['americas', 'baku', 'interlagos', 'miami', 'red_bull_ring', 'rodriguez', 'shanghai', 'silverstone', 'villeneuve']
['americas', 'baku', 'catalunya', 'interlagos', 'miami', 'monaco', 'rodriguez', 'shanghai', 'silverstone', 'suzuka', 'villeneuve', 'zandvoort']
['americas', 'baku', 'catalunya', 'interlagos', 'miami', 'rodriguez', 'shanghai', 'silverstone', 'suzuka', 'villeneuve']
### ['miami'] 8
['baku', '

['albert_park', 'catalunya', 'imola', 'interlagos', 'miami', 'monza', 'rodriguez', 'shanghai', 'silverstone', 'suzuka', 'vegas', 'zandvoort']
['albert_park', 'americas', 'bahrain', 'baku', 'catalunya', 'miami', 'monza', 'rodriguez', 'shanghai', 'silverstone', 'vegas']
['albert_park', 'americas', 'baku', 'interlagos', 'miami', 'red_bull_ring', 'rodriguez', 'shanghai', 'silverstone', 'villeneuve']
['albert_park', 'americas', 'baku', 'catalunya', 'interlagos', 'miami', 'monaco', 'rodriguez', 'shanghai', 'silverstone', 'suzuka', 'villeneuve', 'zandvoort']
['albert_park', 'americas', 'baku', 'catalunya', 'interlagos', 'miami', 'rodriguez', 'shanghai', 'silverstone', 'suzuka', 'villeneuve']
### ['miami'] 8
['albert_park', 'baku', 'catalunya', 'interlagos', 'rodriguez', 'shanghai', 'silverstone', 'spa', 'villeneuve', 'zandvoort']
['albert_park', 'baku', 'catalunya', 'interlagos', 'shanghai', 'silverstone', 'spa', 'suzuka', 'villeneuve', 'zandvoort']
['albert_park', 'catalunya', 'interlagos', 

['albert_park', 'baku', 'catalunya', 'interlagos', 'rodriguez', 'shanghai', 'silverstone', 'spa', 'villeneuve', 'zandvoort']
['albert_park', 'baku', 'catalunya', 'interlagos', 'shanghai', 'silverstone', 'spa', 'suzuka', 'villeneuve', 'zandvoort']
['albert_park', 'catalunya', 'interlagos', 'silverstone', 'suzuka', 'zandvoort']
['albert_park', 'catalunya', 'spa', 'suzuka', 'zandvoort']
['albert_park', 'catalunya', 'interlagos', 'suzuka']
['albert_park', 'interlagos', 'spa', 'suzuka']
['albert_park', 'interlagos']
['albert_park', 'baku', 'marina_bay', 'shanghai', 'suzuka']
['albert_park', 'marina_bay', 'shanghai', 'suzuka']
['albert_park', 'marina_bay', 'suzuka']
['albert_park', 'americas', 'marina_bay']
['albert_park', 'americas']
['americas']
['losail', 'yas_marina']
['bahrain', 'yas_marina']
['yas_marina']
### ['yas_marina'] 24
1 vegas
2 jeddah
3 hungaroring
4 imola
5 monza
6 red_bull_ring
7 monaco
8 miami
9 rodriguez
10 villeneuve
11 silverstone
12 zandvoort
13 catalunya
14 spa
15 int

In [257]:
def nearest_weekend(loc, race, parsed):
    if len(parsed) == 0:
        parsed.append(loc)
    
    race = race + 1
    
    urgent = []
    
    temp = weekend_options.loc[weekend_options['round'] == race, 'circuitRef'].values.tolist()
    
    options = [x for x in temp if x not in parsed]
    
    #return parsed, options
    
    for option in options:
            left = len(weekend_options.loc[(weekend_options['circuitRef'] == option) & (weekend_options['round'] >= race)])
            if left == 1:
                urgent.append(option)
                #next_loc = option
                #return next_loc, distances.loc[loc, next_loc]
    
    #return urgent
    
    print(options)
    
    if len(urgent) > 0:
        print("###", urgent, race)
        options = urgent
    
    next_loc = distances.loc[loc, options].sort_values().head(1).index[0]
    dist = distances.loc[loc, options].sort_values().head(1).values[0]
    
    return next_loc, dist
    
    #dists = distances.loc[loc, options].sort_values().head(2).tail(1)

In [226]:
loc = ''
parsed = []
ships = [0, 0, 0, 0, 0]

cal_cols = range(1, 25)
cal = dict.fromkeys(cal_cols, 0)

start_loc = weekend_options.loc[weekend_options['round'] == 1, 'circuitRef'].values.tolist()

for start in start_loc:
    cal = dict.fromkeys(cal_cols, 0)
    loc = start
    parsed = []
    ships = [0, 0, 0, 0, 0]
    total_dist = 0
    
    for i in weekend_options['round'].unique().tolist():
        cal[i] = loc
            
        parsed.append(loc)
            
        if i == 24:
            break
            
        loc, dist = nearest_flyaway(loc, i, parsed, ships)
        total_dist = total_dist + dist
    
    for calendar in cal:
        print(calendar, cal[calendar])
    print("Total distance :", round(total_dist, 2), "kms")
    print()        

        

1 albert_park 3 [1, 0, 0, 0, 0]
### ['jeddah'] 2
2 jeddah 3 [1, 2, 0, 0, 0]
3 hungaroring 4 [1, 2, 0, 0, 0]
4 imola 4 [1, 2, 0, 0, 0]
5 monza 4 [1, 2, 0, 0, 0]
6 red_bull_ring 5 [0, 2, 0, 0, 0]
7 monaco 5 [0, 0, 0, 0, 0]
### ['miami'] 8
8 miami 5 [8, 0, 0, 0, 0]
9 rodriguez 6 [8, 9, 0, 0, 0]
10 villeneuve 6 [8, 9, 10, 0, 0]
11 silverstone 7 [8, 9, 10, 0, 0]
12 zandvoort 7 [8, 9, 10, 0, 0]
13 catalunya 7 [0, 9, 10, 0, 0]
14 spa 7 [0, 0, 10, 0, 0]
15 interlagos 8 [15, 0, 0, 0, 0]
16 baku 9 [15, 16, 0, 0, 0]
17 shanghai 9 [15, 16, 17, 0, 0]
18 suzuka 9 [15, 16, 17, 18, 0]
19 marina_bay 10 [15, 16, 17, 18, 19]
20 vegas 10 [20, 16, 17, 18, 19]
21 americas 10 [20, 21, 17, 18, 19]
22 losail 11 [20, 21, 22, 18, 19]
23 bahrain 11 [20, 21, 22, 23, 19]
### ['yas_marina'] 24
1 albert_park
2 jeddah
3 hungaroring
4 imola
5 monza
6 red_bull_ring
7 monaco
8 miami
9 rodriguez
10 villeneuve
11 silverstone
12 zandvoort
13 catalunya
14 spa
15 interlagos
16 baku
17 shanghai
18 suzuka
19 marina_bay
20 vegas

8 miami 5 [8, 0, 0, 0, 0]
9 villeneuve 6 [8, 9, 0, 0, 0]
10 silverstone 6 [8, 9, 0, 0, 0]
11 zandvoort 7 [8, 9, 0, 0, 0]
12 spa 7 [8, 9, 0, 0, 0]
13 catalunya 7 [0, 9, 0, 0, 0]
14 interlagos 7 [14, 0, 0, 0, 0]
15 albert_park 8 [14, 15, 0, 0, 0]
16 marina_bay 9 [14, 15, 16, 0, 0]
17 shanghai 9 [14, 15, 16, 17, 0]
18 suzuka 9 [14, 15, 16, 17, 18]
19 baku 10 [19, 15, 16, 17, 18]
20 vegas 10 [19, 20, 16, 17, 18]
21 americas 10 [19, 20, 21, 17, 18]
22 losail 11 [19, 20, 21, 22, 18]
23 bahrain 11 [19, 20, 21, 22, 23]
### ['yas_marina'] 24
1 rodriguez
2 jeddah
3 hungaroring
4 imola
5 monza
6 red_bull_ring
7 monaco
8 miami
9 villeneuve
10 silverstone
11 zandvoort
12 spa
13 catalunya
14 interlagos
15 albert_park
16 marina_bay
17 shanghai
18 suzuka
19 baku
20 vegas
21 americas
22 losail
23 bahrain
24 yas_marina
Total distance : 103221.95 kms

1 shanghai 3 [1, 0, 0, 0, 0]
### ['jeddah'] 2
2 jeddah 3 [1, 2, 0, 0, 0]
3 hungaroring 4 [1, 2, 0, 0, 0]
4 imola 4 [1, 2, 0, 0, 0]
5 monza 4 [1, 2, 0, 0, 0

In [253]:
loc = ''
parsed = []
ships = [0, 0, 0, 0, 0]

cal_cols = range(1, 25)
cal = dict.fromkeys(cal_cols, 0)

start_loc = weekend_options.loc[weekend_options['round'] == 1, 'circuitRef'].values.tolist()

for start in start_loc:
    cal = dict.fromkeys(cal_cols, 0)
    loc = start
    parsed = []
    ships = [0, 0, 0, 0, 0]
    total_dist = 0
    
    for i in weekend_options['round'].unique().tolist():
        cal[i] = loc
            
        parsed.append(loc)
            
        if i == 24:
            break
            
        loc, dist = nearest_flyaway(loc, i, parsed, ships)
        total_dist = total_dist + dist
    
    for calendar in cal:
        print(calendar, cal[calendar])
    print("Total distance :", round(total_dist, 2), "kms")
    print()        

        

1 albert_park 3 [1, 0, 0, 0, 0] 1
### ['jeddah'] 2
['jeddah']
2 jeddah 3 [1, 2, 0, 0, 0] 2
['americas', 'interlagos', 'marina_bay', 'miami', 'rodriguez', 'shanghai', 'suzuka', 'vegas']
3 marina_bay 4 [1, 2, 3, 0, 0] 3
['interlagos', 'miami', 'rodriguez', 'shanghai', 'suzuka', 'vegas']
4 shanghai 4 [1, 2, 3, 4, 0] 4
['americas', 'bahrain', 'baku', 'miami', 'rodriguez', 'vegas']
5 baku 4 [1, 2, 3, 4, 5] 5
['americas', 'interlagos', 'miami', 'rodriguez', 'villeneuve']
6 villeneuve 5 [6, 2, 3, 4, 5] 5
['americas', 'interlagos', 'miami', 'rodriguez', 'suzuka']
7 miami 5 [6, 7, 3, 4, 5] 5
['americas', 'interlagos', 'rodriguez', 'suzuka']
8 americas 5 [6, 7, 8, 4, 5] 5
['interlagos', 'rodriguez']
9 rodriguez 6 [6, 7, 8, 9, 5] 5
['interlagos', 'suzuka']
10 interlagos 6 [6, 7, 8, 9, 10] 5
['suzuka']
11 suzuka 7 [11, 7, 8, 9, 10] 5
['catalunya', 'hungaroring', 'imola', 'monaco', 'red_bull_ring', 'silverstone', 'spa', 'zandvoort']
12 hungaroring 7 [11, 0, 8, 9, 10] 4
['catalunya', 'imola', 'monac

4 shanghai 4 [1, 2, 3, 4, 0] 4
['albert_park', 'americas', 'bahrain', 'baku', 'miami', 'rodriguez', 'vegas']
5 baku 4 [1, 2, 3, 4, 5] 5
['albert_park', 'americas', 'interlagos', 'miami', 'rodriguez', 'villeneuve']
6 villeneuve 5 [6, 2, 3, 4, 5] 5
['albert_park', 'americas', 'interlagos', 'miami', 'rodriguez', 'suzuka']
7 miami 5 [6, 7, 3, 4, 5] 5
['albert_park', 'americas', 'interlagos', 'rodriguez', 'suzuka']
8 americas 5 [6, 7, 8, 4, 5] 5
['albert_park', 'interlagos', 'rodriguez']
9 rodriguez 6 [6, 7, 8, 9, 5] 5
['albert_park', 'interlagos', 'suzuka']
10 interlagos 6 [6, 7, 8, 9, 10] 5
['albert_park', 'suzuka']
11 albert_park 7 [11, 7, 8, 9, 10] 5
['suzuka']
12 suzuka 7 [11, 12, 8, 9, 10] 5
['catalunya', 'hungaroring', 'imola', 'monaco', 'monza', 'red_bull_ring', 'silverstone', 'zandvoort']
13 hungaroring 7 [11, 12, 0, 9, 10] 4
['catalunya', 'imola', 'monaco', 'monza', 'red_bull_ring', 'silverstone', 'spa', 'zandvoort']
14 red_bull_ring 7 [11, 12, 0, 0, 10] 3
['catalunya', 'imola', '

23 bahrain 11 [21, 22, 23, 0, 0] 3
### ['yas_marina'] 24
['yas_marina']
1 shanghai
2 jeddah
3 marina_bay
4 suzuka
5 baku
6 villeneuve
7 miami
8 americas
9 rodriguez
10 interlagos
11 albert_park
12 hungaroring
13 red_bull_ring
14 imola
15 monza
16 monaco
17 spa
18 zandvoort
19 silverstone
20 catalunya
21 vegas
22 losail
23 bahrain
24 yas_marina
Total distance : 104656.27 kms

1 suzuka 3 [1, 0, 0, 0, 0] 1
### ['jeddah'] 2
['jeddah']
2 jeddah 3 [1, 2, 0, 0, 0] 2
['albert_park', 'americas', 'interlagos', 'marina_bay', 'miami', 'rodriguez', 'shanghai', 'vegas']
3 marina_bay 4 [1, 2, 3, 0, 0] 3
['albert_park', 'interlagos', 'miami', 'rodriguez', 'shanghai', 'vegas']
4 shanghai 4 [1, 2, 3, 4, 0] 4
['albert_park', 'americas', 'bahrain', 'baku', 'miami', 'rodriguez', 'vegas']
5 baku 4 [1, 2, 3, 4, 5] 5
['albert_park', 'americas', 'interlagos', 'miami', 'rodriguez', 'villeneuve']
6 villeneuve 5 [6, 2, 3, 4, 5] 5
['albert_park', 'americas', 'interlagos', 'miami', 'rodriguez']
7 miami 5 [6, 7, 3, 

In [252]:
def nearest_flyaway(loc, race, parsed, ships):
    if len(parsed) == 0:
        parsed.append(loc)
    
    race = race + 1
    
    fa = flyaway.loc[flyaway['circuitRef'] == loc, 'type'].values[0]
    if fa == 'flyaway':
        fa = True
    else:
        fa = False
    
    urgent = []
    
    temp = weekend_options.loc[weekend_options['round'] == race, 'circuitRef'].values.tolist()
    
    options = [x for x in temp if x not in parsed]
    
    ship_count = 0
    
    curr_month = weekend_options.loc[weekend_options['round'] == race-1, 'month'].unique().tolist()[0]
    
    for i in range(0, len(ships)):
        if ((race-1)-ships[i]) >= 5:
            ships[i] = 0
            

    
    #return parsed, options
    
    for option in options:
            left = len(weekend_options.loc[(weekend_options['circuitRef'] == option) & (weekend_options['round'] >= race)])
            if left == 1:
                urgent.append(option)
                #next_loc = option
                #return next_loc, distances.loc[loc, next_loc]
    
    #return urgent
    
    if fa == True:
        for i in range(0, len(ships)):
            if (ships[i]) == 0:
                #ships[i] = weekend_options.loc[weekend_options['round'] == race-1, 'month'].unique().tolist()[0]   
                ships[i] = race-1
                break
                
    for ship in ships:
        if ship > 0:
            ship_count = ship_count + 1
    
    print(race-1, loc, curr_month, ships, ship_count)
        
    
    if len(urgent) > 0:
        print("###", urgent, race)
        options = urgent
        
    else : 
        flyaway_options = [option for option in options if flyaway.loc[flyaway['circuitRef'] == option, 'type'].values[0] == 'flyaway']
        if (len((flyaway_options)) > 0) & (ship_count <= 5) & (fa == True):
            options = flyaway_options
    
    
    print(options)
        
    next_loc = distances.loc[loc, options].sort_values().head(1).index[0]
    dist = distances.loc[loc, options].sort_values().head(1).values[0]
    
    return next_loc, dist
    
    #dists = distances.loc[loc, options].sort_values().head(2).tail(1)

In [244]:
ships

[19, 22, 23, 0, 0]

In [196]:
flyaway.loc[flyaway['circuitRef'] == 'bahrain', 'type'].values[0]

'flyaway'

In [206]:
ship_count = 0
ships = [3, 3, 4, 0, 0]

for ship in ships:
    if ship > 0:
        ship_count = ship_count + 1
        
print(ship_count, 5-ship_count, ships[ship_count+1])

3 2 0


In [204]:
weekend_options.loc[weekend_options['round'] == 7, 'month'].unique().tolist()[0]

5

In [228]:
[option for option in options if flyaway.loc[flyaway['circuitRef'] == option, 'type'].values[0] == 'flyaway']

['albert_park',
 'americas',
 'bahrain',
 'interlagos',
 'jeddah',
 'losail',
 'marina_bay',
 'miami',
 'rodriguez',
 'shanghai',
 'suzuka',
 'vegas',
 'yas_marina']

In [229]:
options

['albert_park',
 'americas',
 'bahrain',
 'catalunya',
 'interlagos',
 'jeddah',
 'losail',
 'marina_bay',
 'miami',
 'rodriguez',
 'shanghai',
 'suzuka',
 'vegas',
 'yas_marina']

In [254]:
len([])

0